# Importing necessary packages

In [1]:
import scala.io.Source
import scala.util.control.Breaks._
import scala.util.matching.Regex 
import org.apache.spark.rdd.RDD
import java.io._
import org.apache.spark.graphx._
import org.apache.spark.SparkContext._
import org.apache.spark.SparkConf
import org.apache.spark.SparkContext
import org.apache.spark.rdd.RDD
import scala.util.MurmurHash
import org.apache.hadoop.conf._
import org.apache.hadoop.io._
import org.apache.hadoop.mapreduce.lib.input._

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.8:4040
SparkContext available as 'sc' (version = 2.4.4, master = local[*], app id = local-1575544608718)
SparkSession available as 'spark'


import scala.io.Source
import scala.util.control.Breaks._
import scala.util.matching.Regex
import org.apache.spark.rdd.RDD
import java.io._
import org.apache.spark.graphx._
import org.apache.spark.SparkContext._
import org.apache.spark.SparkConf
import org.apache.spark.SparkContext
import org.apache.spark.rdd.RDD
import scala.util.MurmurHash
import org.apache.hadoop.conf._
import org.apache.hadoop.io._
import org.apache.hadoop.mapreduce.lib.input._


# Taking the input pair file

In [23]:
val input = sc.textFile("pairs2.txt").map(i=>i.split("\\s+"))
val pairInput = input.map(x=>(x(0),x(1)))

input: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[104] at map at <console>:57
pairInput: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[105] at map at <console>:58


# hashing the input

In [3]:
    val zipPair = pairInput.map(_._1).distinct.zipWithIndex.collect.toMap
    val zipPair2 = pairInput.map(_._2).distinct.zipWithIndex.collect.toMap
    val pairInputHash = pairInput.map{case(v1,v2)=>(zipPair.get(v1),zipPair2.get(v2))}
    val cleanInput = pairInputHash.map{case(Some(v1),Some(v2))=>(v1,v2)}

<console>:57: warning: match may not be exhaustive.
It would fail on the following inputs: (None, None), (None, Some(_)), (Some(_), None)
           val cleanInput = pairInputHash.map{case(Some(v1),Some(v2))=>(v1,v2)}
                                             ^
zipPair: scala.collection.immutable.Map[String,Long] = Map(5390980720f70186a0e02cb9 -> 402735, 5390b5df20f70186a0f0b7da -> 105409, 5390aaf920f70186a0eaefb7 -> 294191, 53908b2a20f70186a0db8f14 -> 670846, 5390a77d20f70186a0e8e8c9 -> 576295, 53909f6920f70186a0e3a197 -> 508660, 5390aaf920f70186a0eaeca1 -> 686090, 559253ae0cf28b1a968ffc09 -> 702136, 5390b3da20f70186a0ef5e07 -> 791700, 558b3498612c41e6b9d4697e -> 898659, 5390b4c420f70186a0eff128 -> 621443, 5390a0b720f70186a0e4e4fd -> 102400, 5390a0b720f70186a0e50068 -> 6035, 53908b4920f70186a0dbcb40 -> 725862, 53908adf20f70186a0daca84 -> 317986, 53909f2d20f70186a0e3920b -> 869840, 5390bf1320f70186a0f510c3 -> 334928, 5390981d20f70186a0e05979 -> 476586, 53908b1820f70186a0db39ba -> 20

# Making the graph

In [4]:
    val edges: RDD[(VertexId,VertexId)] = cleanInput
    val graph = Graph.fromEdgeTuples(edges,null)
    

edges: org.apache.spark.rdd.RDD[(org.apache.spark.graphx.VertexId, org.apache.spark.graphx.VertexId)] = MapPartitionsRDD[15] at map at <console>:57
graph: org.apache.spark.graphx.Graph[Null,Int] = org.apache.spark.graphx.impl.GraphImpl@2e044de


# Calculating Inlinks,outlink and No. of traingle

In [5]:
val countTriangles = graph.triangleCount
val inlinks = graph.inDegrees
val outlinks = graph.outDegrees

countTriangles: org.apache.spark.graphx.Graph[Int,Int] = org.apache.spark.graphx.impl.GraphImpl@69c193ad
inlinks: org.apache.spark.graphx.VertexRDD[Int] = VertexRDDImpl[81] at RDD at VertexRDD.scala:57
outlinks: org.apache.spark.graphx.VertexRDD[Int] = VertexRDDImpl[85] at RDD at VertexRDD.scala:57


In [6]:
val cntTraingle=countTriangles.vertices.collect()

cntTraingle: Array[(org.apache.spark.graphx.VertexId, Int)] = Array((682981,0), (354796,0), (755508,0), (194402,0), (754897,0), (846729,1), (101257,0), (165737,0), (934765,0), (619983,0), (735956,0), (648817,0), (814762,0), (447187,0), (985335,0), (237445,0), (903110,0), (857454,0), (904930,0), (720200,0), (161980,0), (980434,0), (363818,0), (422955,0), (308958,0), (162877,0), (581139,0), (547183,0), (4992,0), (571636,0), (853554,0), (585442,0), (260793,0), (496158,0), (813189,0), (941213,0), (411294,0), (116662,0), (851916,0), (442260,0), (551291,0), (928616,0), (245388,0), (192192,0), (584766,0), (316030,0), (61139,0), (9867,0), (355173,0), (683020,0), (519922,0), (167635,0), (272285,0), (992030,0), (748293,0), (559624,0), (774852,0), (459277,0), (451607,0), (969423,0), (299897,0), (5...

In [7]:
val pw = new PrintWriter(new File("cntTraingle.txt" ))
cntTraingle.foreach(x=>pw.write(x.toString.substring(1,x.toString.length()-1)+"\n"))
pw.close()  

pw: java.io.PrintWriter = java.io.PrintWriter@29624b31


# making outlinks and inlinks as array

In [8]:
val out=outlinks.collect()
val in=inlinks.collect()

out: Array[(org.apache.spark.graphx.VertexId, Int)] = Array((682981,3), (354796,14), (755508,1), (194402,14), (754897,6), (846729,2), (101257,14), (165737,9), (619983,14), (735956,4), (648817,6), (814762,6), (447187,7), (237445,12), (903110,9), (857454,4), (904930,19), (720200,2), (161980,10), (363818,13), (422955,10), (308958,22), (162877,15), (581139,4), (547183,11), (4992,4), (571636,20), (853554,12), (585442,1), (260793,6), (496158,4), (813189,34), (411294,5), (116662,12), (851916,19), (442260,18), (551291,1), (245388,23), (192192,9), (584766,21), (316030,1), (61139,6), (9867,2), (355173,19), (683020,4), (519922,4), (167635,3), (272285,1), (748293,22), (559624,16), (774852,1), (459277,13), (451607,1), (299897,18), (511472,4), (591825,5), (754715,3), (2938,20), (125957,2), (210106,5)...

In [9]:
val x = in.zip(out)

x: Array[((org.apache.spark.graphx.VertexId, Int), (org.apache.spark.graphx.VertexId, Int))] = Array(((682981,20),(682981,3)), ((354796,7),(354796,14)), ((755508,4),(755508,1)), ((194402,2),(194402,14)), ((754897,4),(754897,6)), ((846729,206),(846729,2)), ((101257,1),(101257,14)), ((165737,5),(165737,9)), ((934765,6),(619983,14)), ((619983,1),(735956,4)), ((735956,1),(648817,6)), ((648817,1),(814762,6)), ((814762,1),(447187,7)), ((447187,14),(237445,12)), ((985335,1),(903110,9)), ((237445,38),(857454,4)), ((903110,7),(904930,19)), ((857454,2),(720200,2)), ((904930,1),(161980,10)), ((720200,4),(363818,13)), ((161980,3),(422955,10)), ((980434,7),(308958,22)), ((363818,49),(162877,15)), ((422955,2),(581139,4)), ((308958,13),(547183,11)), ((162877,2),(4992,4)), ((581139,113),(571636,20)), (...

# calculating degree of each vertex

In [10]:
val cleanInOut = x.map{case((p1,in),(duplicate_p1,out))=>(p1,out+in)}

cleanInOut: Array[(org.apache.spark.graphx.VertexId, Int)] = Array((682981,23), (354796,21), (755508,5), (194402,16), (754897,10), (846729,208), (101257,15), (165737,14), (934765,20), (619983,5), (735956,7), (648817,7), (814762,8), (447187,26), (985335,10), (237445,42), (903110,26), (857454,4), (904930,11), (720200,17), (161980,13), (980434,29), (363818,64), (422955,6), (308958,24), (162877,6), (581139,133), (547183,13), (4992,4), (571636,7), (853554,6), (585442,44), (260793,16), (496158,30), (813189,20), (941213,19), (411294,3), (116662,24), (851916,10), (442260,23), (551291,19), (928616,17), (245388,8), (192192,25), (584766,5), (316030,8), (61139,20), (9867,2), (355173,24), (683020,20), (519922,6), (167635,14), (272285,3), (992030,20), (748293,5), (559624,8), (774852,6), (459277,23), ...

# Saving Vertex degree to file

In [11]:
val pw = new PrintWriter(new File("vertexdeg.txt" ))
cleanInOut.foreach(x=>pw.write(x.toString.substring(1,x.toString.length()-1)+"\n"))
pw.close()  

pw: java.io.PrintWriter = java.io.PrintWriter@55ee90ac


In [12]:
val vdegT = sc.textFile("vertexdeg.txt").map(i=>i.split(","))
val vdeg = vdegT.map(x=>(x(0),x(1).toDouble))

vdegT: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[88] at map at <console>:55
vdeg: org.apache.spark.rdd.RDD[(String, Double)] = MapPartitionsRDD[89] at map at <console>:56


In [13]:
val cntTraingle = sc.textFile("cntTraingle.txt").map(i=>i.split(","))
val cntT = cntTraingle.map(x=>(x(0),x(1).toDouble))

cntTraingle: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[92] at map at <console>:57
cntT: org.apache.spark.rdd.RDD[(String, Double)] = MapPartitionsRDD[93] at map at <console>:58


# combining traingle and degree count 

In [14]:
val x = cntT.zip(vdeg)

x: org.apache.spark.rdd.RDD[((String, Double), (String, Double))] = ZippedPartitionsRDD2[94] at zip at <console>:56


In [15]:
x.take(10).foreach(println)

((682981,0.0),(682981,23.0))
((354796,0.0),(354796,21.0))
((755508,0.0),(755508,5.0))
((194402,0.0),(194402,16.0))
((754897,0.0),(754897,10.0))
((846729,1.0),(846729,208.0))
((101257,0.0),(101257,15.0))
((165737,0.0),(165737,14.0))
((934765,0.0),(934765,20.0))
((619983,0.0),(619983,5.0))


In [16]:
val z = x.map{case((p1,a),(duplicate_p1,b))=>(p1,a,b)}

z: org.apache.spark.rdd.RDD[(String, Double, Double)] = MapPartitionsRDD[95] at map at <console>:54


In [17]:
z.take(10).foreach(println)

(682981,0.0,23.0)
(354796,0.0,21.0)
(755508,0.0,5.0)
(194402,0.0,16.0)
(754897,0.0,10.0)
(846729,1.0,208.0)
(101257,0.0,15.0)
(165737,0.0,14.0)
(934765,0.0,20.0)
(619983,0.0,5.0)


# calculating local cluster

In [18]:
val localcluster=z.map{case(v,t,k)=>(v,t/(k*(k-1)/2))}

localcluster: org.apache.spark.rdd.RDD[(String, Double)] = MapPartitionsRDD[96] at map at <console>:54


In [19]:
localcluster.foreach(println)

(682981,0.0)
(354796,0.0)
(755508,0.0)
(194402,0.0)
(752277,0.0)
(754897,0.0)
(222891,0.0)
(505589,0.0)
(306039,0.0)
(731945,0.0)
(846729,4.645113340765515E-5)
(101257,0.0)
(165737,0.0)
(934765,0.0)
(72260,0.0)
(619983,0.0)
(368790,0.0)
(735956,0.0)
(49458,0.0)
(648817,0.0)
(814762,0.0)
(447187,0.0)
(929194,0.0)
(985335,0.0)
(945262,0.0)
(237445,0.0)
(781449,0.0)
(903110,0.0)
(516847,0.0)
(857454,0.0)
(198659,0.0)
(904930,0.0)
(720200,0.0)
(161980,0.0)
(980434,0.0)
(363818,0.0)
(951996,0.0016806722689075631)
(422955,0.0)
(283146,0.0)
(308958,0.0)
(502807,0.0)
(162877,0.0)
(959432,0.0)
(581139,0.0)
(499076,0.0)
(547183,0.0)
(98585,0.0)
(4992,0.0)
(138703,0.0)
(571636,0.0)
(508891,0.0)
(853554,0.0)
(987564,0.0)
(585442,0.0)
(394387,0.0)
(260793,0.0)
(44427,0.0)
(496158,0.0)
(488416,0.0)
(813189,0.0)
(885631,0.0)
(941213,0.0)
(559773,0.0)
(411294,0.0)
(1003060,0.0)
(116662,0.0)
(821112,0.0)
(851916,0.0)
(644390,0.0)
(442260,0.0)
(229144,0.0)
(551291,0.0)
(603583,0.0)
(928616,0.0)
(583784,

(202085,0.0)
(990606,0.0)
(739069,0.0)
(420537,0.1)
(788976,0.0)
(463606,0.0)
(995637,0.0)
(447135,0.0)
(50810,0.0)
(694908,0.0)
(849329,0.0)
(169383,0.0)
(765466,0.0)
(446998,0.0)
(417560,0.0)
(436117,0.0)
(783042,0.0)
(678021,0.0)
(408460,0.0)
(750678,0.0)
(943020,0.0)
(406230,0.0)
(282217,0.0)
(205328,0.0)
(618839,0.0)
(870070,0.0)
(641524,0.0)
(885475,0.0)
(190723,0.0)
(12109,0.0)
(544908,0.0)
(868224,0.0)
(259974,0.0)
(191347,0.0)
(466862,0.0)
(354185,0.0)
(141550,0.0)
(578864,0.0)
(102979,0.0)
(79300,0.0)
(240636,0.0)
(491530,0.0)
(366658,0.0)
(729976,0.0)
(506031,0.0)
(720928,0.0)
(878663,0.0)
(694369,0.0)
(244055,0.0)
(986505,0.0)
(267949,0.0)
(24505,0.0)
(338487,0.0)
(147082,0.0)
(797569,0.0)
(107263,0.0)
(715942,0.0)
(531011,0.0)
(935954,0.0)
(851747,0.0)
(487701,0.0)
(785616,0.0)
(94100,0.0)
(486486,0.0)
(405606,0.0)
(28431,0.0)
(177690,0.0)
(755365,0.0)
(310979,0.0)
(496925,0.0)
(783152,0.0)
(57980,0.0)
(862088,0.0)
(513272,0.0)
(905430,0.0)
(744607,0.0)
(912450,0.0)
(49115

(315165,0.0)
(634413,0.0)
(951840,0.0)
(790413,0.0)
(809022,0.0)
(571948,0.0)
(487649,0.0)
(681863,0.0)
(32922,0.0)
(287638,0.0)
(779746,0.0)
(598214,0.0)
(554274,0.0)
(53644,0.06666666666666667)
(685808,0.0)
(157488,0.0)
(400861,0.0)
(978529,0.0)
(104221,8.378367056260735E-5)
(403702,0.0)
(916019,0.0)
(569907,0.0)
(949403,0.0)
(65676,0.0)
(73716,4.522840343735866E-4)
(652236,0.0)
(55997,0.0)
(79014,0.0)
(599358,0.0)
(861302,0.0)
(9470,0.0)
(891481,0.0)
(485257,0.0)
(932652,0.0)
(441577,0.0)
(613216,0.0)
(937267,0.0)
(590453,0.0)
(803757,0.0)
(936175,0.0)
(851194,0.0)
(312065,0.0)
(5921,0.047619047619047616)
(390156,0.0)
(371741,0.0)
(100048,0.0)
(601399,0.0)
(668434,0.0)
(114146,0.0)
(258960,0.0)
(42672,0.0)
(898729,0.0)
(624903,0.0)
(418444,0.0)
(303608,0.0)
(376948,0.0)
(258823,0.0)
(781768,0.0)
(414511,0.0)
(575490,0.0)
(857720,0.0)
(558590,0.0)
(415798,0.0)
(282392,0.0)
(388537,0.0)
(382258,0.0)
(167628,0.0)
(448337,0.0)
(708688,0.0)
(11706,0.0)
(897110,0.0)
(786857,0.0)
(778420,0

(23094,0.0)
(491224,0.0)
(187902,3.390060343074107E-5)
(785648,0.0)
(912951,0.07142857142857142)
(626528,0.0)
(355836,0.0)
(410884,0.0)
(434785,0.0)
(705666,0.0)
(506064,0.0)
(988481,0.0)
(896623,0.0)
(554697,0.0)
(382603,0.0)
(191880,0.0)
(279422,0.0)
(940784,0.0)
(722943,0.0)
(760032,0.0)
(525993,0.0)
(691808,0.0)
(621127,0.0)
(574847,0.0)
(321483,0.0)
(11193,0.0)
(350291,0.0)
(133887,0.0)
(408739,0.0)
(519805,0.0)
(909681,0.0)
(31122,0.0)
(519902,0.0)
(18746,0.0)
(495553,0.0)
(690677,0.0)
(531537,0.0)
(182195,0.0)
(623330,0.0)
(878605,0.0)
(244588,0.0)
(727597,0.0)
(730424,0.0)
(307957,0.0)
(839806,0.0)
(270205,0.0022988505747126436)
(293923,0.0)
(73333,0.0)
(856914,0.0)
(655122,0.0)
(279955,0.0)
(504614,0.06666666666666667)
(733096,0.0)
(781358,0.0)
(874926,0.0)
(1003788,0.0)
(117182,0.0)
(464119,0.0)
(77792,0.0)
(146029,0.0)
(92820,0.0)
(263783,0.0)
(666913,0.0)
(598403,0.0)
(864864,0.0)
(972725,0.0)
(13468,0.0)
(655330,0.0)
(483470,0.0)
(180752,0.0)
(787098,0.0)
(537225,0.0)
(666

(710963,0.0)
(1274,0.0)
(822893,0.0)
(540917,0.0)
(553689,0.0)
(148707,0.0)
(928973,0.0)
(26364,0.0)
(504380,0.0)
(364208,0.0)
(14033,0.0)
(189241,0.0)
(790601,0.0)
(210756,0.0)
(672457,0.0)
(6812,0.0)
(609095,0.0)
(716313,0.0)
(1002800,0.0)
(100789,0.0)
(523360,0.0)
(531037,0.0)
(549802,0.0)
(660972,0.0)
(466810,0.0)
(944859,0.0)
(959172,0.0)
(453290,0.0)
(610005,0.0)
(952282,0.0)
(335452,0.0)
(383584,0.0)
(491868,0.0)
(310264,0.0)
(582114,0.0)
(846215,0.0)
(46332,0.0)
(647263,0.0)
(976963,0.0)
(924267,0.0)
(1005745,0.0)
(440550,0.0)
(298629,0.0)
(162870,0.0)
(54223,0.00946969696969697)
(229014,0.0)
(20826,0.0)
(767890,0.0)
(377702,0.0)
(213791,0.0)
(546572,0.0)
(76745,0.0)
(937781,0.0)
(75354,0.0)
(239473,0.0)
(640620,0.0)
(687453,0.0)
(1005361,0.0)
(676689,0.0)
(421921,0.0)
(259220,0.0)
(720869,0.0)
(632749,0.0)
(763587,0.0)
(215000,0.0)
(376278,0.0)
(684027,0.0)
(333294,0.0)
(878494,0.0)
(419016,0.0)
(344454,0.0)
(591123,0.0)
(900919,0.0)
(17225,0.0)
(706712,0.0)
(675909,0.0)
(4569

...

(568319,0.0)
(28429,0.0)
(772257,0.0)
(841053,0.0)
(566896,0.0)
(723747,0.0)
(786303,0.0)
(478372,0.0)
(897609,0.0)
(818017,0.0)
(828937,0.0)
(338707,0.0)
(403037,0.0)
(235935,0.0)
(599123,0.0)
(809372,0.0)
(109062,0.0)
(921003,0.0)
(311582,0.0)
(373527,0.0)
(122328,0.0)
(551283,0.0)
(139586,0.0)
(756813,0.0)
(716272,0.0)
(89984,0.0)
(363530,0.0)
(804588,0.0)
(811517,0.0)
(1002285,0.0)
(129621,0.0)
(555891,0.0)
(821351,0.0)
(312505,0.0)
(857023,0.0)
(373690,0.0)
(17529,0.0)
(673080,0.0)
(259218,0.0)
(924350,0.0)
(604797,0.0)
(986425,0.0)
(873546,0.0)
(595944,0.0)
(499250,0.0)
(441660,0.0)
(632487,0.0)
(781688,0.0)
(305505,0.0)
(461284,0.0)
(171228,0.0)
(82022,0.0)
(300740,0.0)
(25166,0.0)
(861203,0.0)
(529716,0.0026455026455026454)
(906014,0.0)
(39044,0.0)
(154724,0.0)
(776501,0.0)
(579889,0.0)
(31341,0.0)
(281065,0.0)
(846916,0.0)
(325239,0.0)
(805725,0.0)
(982278,0.0)
(16638,0.0)
(121301,0.0)
(852909,0.0)
(334716,0.0)
(982415,0.0)
(112877,0.0)
(55326,0.0)
(932904,0.0)
(965112,0.0)
(4

(82483,0.0)
(907385,0.0)
(868912,0.0)
(265218,0.0)
(388042,0.0)
(911584,0.0)
(118181,0.0)
(294045,0.0)
(9534,0.0)
(996182,0.0)
(349802,0.0)
(810470,0.0)
(957175,0.0)
(488974,0.0)
(11575,0.0)
(573500,0.0)
(602438,0.0)
(265107,0.0)
(313181,0.0)
(149388,0.0)
(796762,0.0)
(872214,0.0)
(940613,0.0)
(779322,0.0)
(267246,0.0)
(267350,0.0)
(80813,0.0)
(978853,0.0)
(466802,0.0)
(173652,0.0)
(183636,0.0)
(330140,0.0)
(448583,0.0)
(746627,0.0)
(74969,0.0)
(666092,0.0)
(285881,0.0)
(179340,0.0)
(730254,0.0)
(430103,0.0)
(59525,0.0)
(616985,0.0)
(270353,0.0)
(8819,0.0)
(184949,1.3333333333333333)
(958371,0.0)
(77732,0.0)
(795891,0.0)
(78629,0.0)
(232861,0.0)
(268273,0.0)
(561787,0.0)
(102750,0.0)
(418624,0.0)
(939970,0.0)
(879988,0.0)
(369868,0.0)
(731801,0.0)
(673366,0.0)
(864329,0.0)
(223481,0.0)
(29118,0.0)
(520278,0.0)
(717748,0.0)
(985444,0.0)
(800428,0.0)
(343198,0.0)
(10307,0.0)
(617505,0.0)
(125065,0.0)
(238308,0.0)
(759933,0.0)
(758633,0.0)
(966964,0.0)
(910024,0.0)
(651799,0.0)
(993959,0.

(303711,0.0)
(485555,0.0)
(355535,0.0)
(65882,0.0)
(567182,0.0)
(654854,0.0)
(536703,0.0)
(354352,0.0)
(487524,0.0)
(212841,0.0)
(658078,0.0)
(1000361,0.0)
(22150,0.0)
(968271,0.0)
(660990,0.0)
(85941,0.0)
(949960,0.0)
(785660,0.0)
(92188,0.0)
(621567,0.0)
(71433,0.0)
(528637,0.0)
(246641,0.0)
(131623,0.0)
(967523,0.0)
(415420,0.0)
(301410,0.0)
(742688,0.0)
(193958,0.0)
(95048,0.0)
(296080,0.0)
(822970,0.0)
(785978,0.0)
(301910,0.0)
(925657,0.0)
(124870,0.0)
(453178,0.0)
(533583,0.0)
(691059,0.0)
(619364,0.0)
(993549,0.0)
(660151,0.0)
(189376,0.0)
(830601,0.0)
(175797,0.0)
(320656,0.0)
(994102,0.0)
(898955,0.0)
(562807,0.0)
(205918,0.0)
(473387,0.0)
(608886,0.0)
(903693,0.0)
(65973,0.0)
(508799,0.0)
(57387,0.13333333333333333)
(175726,0.0)
(79103,0.0)
(673437,0.0)
(435342,0.0)
(206763,0.0)
(123674,0.0)
(783827,0.0)
(374080,0.0)
(334053,0.0)
(151513,0.0)
(994394,0.0)
(61703,0.0)
(123713,0.0)
(867124,0.0)
(766686,0.0)
(614788,0.0)
(913723,0.0)
(793349,0.0)
(786706,0.0)
(653398,0.0)
(4938

(568033,0.0)
(670109,0.0)
(513082,0.03571428571428571)
(898753,0.0)
(247798,0.0)
(778250,0.0)
(470130,0.0)
(972983,0.0)
(390973,0.0)
(340592,0.0017825311942959)
(644025,0.0)
(818160,0.0)
(177903,0.0)
(33330,0.0)
(483897,0.0)
(922654,0.0)
(528988,0.0)
(846825,0.0)
(252380,0.0)
(120040,0.0)
(811907,0.0)
(369907,0.0)
(943207,0.0)
(610881,0.0)
(586506,0.0)
(518126,0.0)
(862672,0.0)
(619117,0.0)
(691969,0.010526315789473684)
(178911,0.0)
(692359,0.0)
(797308,0.0)
(610647,0.0)
(893930,0.0)
(996195,0.0)
(493264,0.0)
(614066,0.0)
(471476,0.0)
(95711,0.0)
(225288,0.0)
(791178,0.0)
(87319,0.0)
(348099,0.0)
(986016,0.0)
(486985,0.0)
(394041,0.0)
(956538,0.0)
(202785,0.0)
(733302,0.0)
(268897,0.0)
(974524,0.0)
(631194,0.0)
(210904,0.0)
(299713,0.0)
(761460,0.0)
(600982,0.0)
(167666,0.0)
(706301,0.0)
(366286,0.0)
(700120,0.0)
(678488,0.0)
(715141,0.0)
(175056,0.0)
(680886,0.0)
(272556,0.0)
(1006055,0.0)
(606084,0.0)
(35196,0.0)
(235058,0.0)
(422017,0.0)
(639442,0.0)
(955960,0.0)
(707998,0.0)
(31777

org.apache.spark.SparkException:  Job aborted due to stage failure: Task 0 in stage 24.0 failed 1 times, most recent failure: Lost task 0.0 in stage 24.0 (TID 234, localhost, executor driver): org.apache.spark.SparkException: Can only zip RDDs with same number of elements in each partition

In [20]:
val lcluster=localcluster.map{case(v,c)=>(v,c)}

lcluster: org.apache.spark.rdd.RDD[(String, Double)] = MapPartitionsRDD[97] at map at <console>:54


# writing local clusters to file

In [26]:
val pw = new PrintWriter(new File("localCC.txt" ))
lcluster.foreach(x=>pw.write(x.toString.substring(1,x.toString.length()-1)+"\n"))
pw.close()  

pw: java.io.PrintWriter = java.io.PrintWriter@11a59cf1


In [28]:
val clstr = sc.textFile("localCC.txt").map(i=>i.split(","))
val clstr2 = clstr.map(x=>(x(0),x(1).toDouble))

clstr: org.apache.spark.rdd.RDD[Array[String]] = MapPartitionsRDD[108] at map at <console>:55
clstr2: org.apache.spark.rdd.RDD[(String, Double)] = MapPartitionsRDD[109] at map at <console>:56


In [29]:
val z=clstr2.toDF

z: org.apache.spark.sql.DataFrame = [_1: string, _2: double]


In [30]:
z.createOrReplaceTempView("cluster")

# calculating average of cluster

In [31]:
val avg =spark.sql("select mean(_2) from cluster")

avg: org.apache.spark.sql.DataFrame = [avg(_2): double]


In [33]:
val avgrdd = avg.rdd.collect()

avgrdd: Array[org.apache.spark.sql.Row] = Array([0.0017568950402310996])


# writing Average cluster to file

In [34]:
avgrdd.foreach(println)
val pw = new PrintWriter(new File("AverageCC.txt" ))
avgrdd.foreach(x=>pw.write(x.toString)+"\n")
pw.close()  

[0.0017568950402310996]


pw: java.io.PrintWriter = java.io.PrintWriter@1fb07ecf
